In [1]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
#from pyscf.geomopt.berny_solver import optimize
#from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
from pyscf.scf._response_functions import _gen_rhf_response 

In [2]:
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [3]:
mol = gto.M(atom='H 0 0 0; H 0 0 1.2', unit="Bohr",basis="sto-3g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -1.11033388268018


In [4]:
C=mf.mo_coeff
P=mf.make_rdm1()
O=np.diag(mf.mo_occ)
mo_coeff=mf.mo_coeff
mo_occ=mf.mo_occ
mo_energy=mf.mo_energy

In [5]:
mfa=fc(mf,[.01,-.01])
mfa.run()
P1=(mfa.make_rdm1()-P)
#C1=(mfa.mo_coeff*[1,-1]-C)
C1=mfa.mo_coeff-C

converged SCF energy = -1.11050145845196


In [6]:
hs=mf.Hessian()

In [7]:
#fx is a function  fx(dm_mo) => v1_mo.  A function to generate the induced potential.
#also passes through  pyscf.scf._response_functions._gen_rhf_response
fxh=pyscf.hessian.rhf.gen_vind(mf,mf.mo_coeff,mf.mo_occ)
#h1vo come from h1ao generated by the function: is a list with an element per each atom
h1ao=hs.make_h1(mf.mo_coeff,mf.mo_occ)
#s1ao comes from the espextation value of the nabla operator
s1ao=mol.intor('int1e_ipovlp', comp=3).shape        

In [8]:
from pyscf.prop.polarizability.rhf import Polarizability
plo=Polarizability(mf)
fxp=plo.gen_vind(mf,mf.mo_coeff,mf.mo_occ)

In [9]:
nao, nmo = mo_coeff.shape
mocc = mo_coeff[:,mo_occ>0] 
nocc = mocc.shape[1]
vresp = _gen_rhf_response(mf, mo_coeff, mo_occ, hermi=1)

In [10]:
mf.mo_coeff,mf.mo_occ

(array([[ 0.53781331,  1.35743672],
        [ 0.53781331, -1.35743672]]),
 array([2., 0.]))

In [11]:
nocc

1

In [12]:
mo_occ

array([2., 0.])

In [13]:
h1mo=DeltaV(mol,[.01,-.01])
h1vo=(C@h1mo@C.T)
#h1vo = np.vstack(h1vo)
h1vo

array([[ 0.00775226, -0.01063922],
       [-0.01063922,  0.00775226]])

In [14]:
import numpy
from pyscf import lib

def solve_nos1(fvind, mo_energy, mo_occ, h1,max_cycle=20, tol=1e-9, hermi=False):
    '''For field independent basis. First order overlap matrix is zero'''
    e_a = mo_energy[mo_occ==0]
    e_i = mo_energy[mo_occ>0]
    e_ai = 1 / lib.direct_sum('a-i->ai', e_a, e_i)
    mo1base = h1 * -e_ai

    def vind_vo(mo1):
        v = fvind(mo1.reshape(h1.shape)).reshape(h1.shape)
        v *= e_ai
        return v.ravel()
    mo1 = lib.krylov(vind_vo, mo1base.ravel(),tol=tol, max_cycle=max_cycle, hermi=hermi)
    return mo1.reshape(h1.shape), None


In [15]:
mo_energy, mf.mo_occ

(array([-0.624462  ,  0.79184861]), array([2., 0.]))

In [16]:
dC=solve_nos1(fxh, mf.mo_energy ,mf.mo_occ,h1vo[nocc:,:])[0]
print(dC)

[[ 0.00379187 -0.00615011]]


In [17]:
s1 = numpy.zeros_like(h1vo)
dC=cphf.solve(fxp, mo_energy ,mf.mo_occ,h1vo,s1)[0]
dC

ValueError: cannot reshape array of size 2 into shape (1,1)

In [ ]:
mf.mo_coeff

In [18]:
mfa=fc(mf,[.01,-.01])
mfa.run()
mfa.mo_coeff

converged SCF energy = -1.11050145845196


array([[ 0.54565105, -1.35430518],
       [ 0.5299576 ,  1.36052289]])

In [23]:
(mfa.mo_coeff-mf.mo_coeff)[:,0]

array([ 0.00783774, -0.00785571])